In [1]:
import time
import numpy as np
import pandas as pd

In [2]:
import tensorflow.python.keras as keras
from tensorflow.python.keras.layers import Layer
import dan_models
import dan_utils

In [3]:
randseed = 25
res = 11

v = pd.read_csv('../data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
det_with_class = pd.read_csv('../res/%i_res%i_id_402_withclass.csv'%(randseed, res), index_col=0)

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = det_with_class[det_with_class['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

num_class = det_with_class['class_i'].drop_duplicates().size

v_class = []
for i in range(num_class):
    v_class.append(v[v['class_i']==i])

print('There are %i class(es)'%num_class)

dist_mat = pd.read_csv('../data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('../data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])

seg = pd.read_csv('../data/segement.csv', header=None)
num_dets = 25

det_list_class = []
for i in range(num_class):
    det_list_class_temp, v_class_temp = dan_utils.get_class_with_node(seg, v_class[i])
    det_list_class.append(det_list_class_temp)
    v_class_temp = v_class_temp[v_class_temp['id'].isin(det_list_class_temp[:num_dets])]
    v_class[i] = v_class_temp
    
near_road_set = []
for i in range(num_class):
    near_road_set.append(dan_utils.rds_mat(dist_mat, det_list_class[i][:num_dets], seg))

There are 5 class(es)


C:\Users\10169\.conda\envs\dan_traff\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [4]:
# ind, class
# 0  , blue
# 1  , green
# 2  , yellow  <--
# 3  , black   <--
# 4  , red     <--
class_color_set = ['b', 'g', 'y', 'black', 'r']
class_i = 3

near_road = np.array(near_road_set[class_i])
flow = v_class[class_i].iloc[:, 2:-1]

prop = 24/31  # proportion of training data

k, t_p, t_input, t_pre, num_links = 5, 10, 12, 3, 25
from_day = 1
to_day = 31

image_train, image_test, day_train, day_test, label_train, label_test\
= dan_utils.sliding_window(
    flow, near_road, from_day, to_day, prop, 
    k, t_p, t_input, t_pre, num_links
)

(25, 4464)
(1161, 5, 12, 25)
(898, 5, 12, 25)
(263, 5, 12, 25)
(898, 25, 3)
(263, 25, 3)


In [5]:
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

finish_model = dan_models.build_model(input_data, input_HA)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         [(None, 5, 12, 25)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 5, 12, 25)    100         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 5, 12, 25)    5650        batch_normalization[0][0]        
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 5, 12, 25)    0           conv2d[0][0]                     
______________________________________________________________________________________________

In [6]:
finish_model.compile(optimizer='adam', loss='mean_squared_error')

In [7]:
X_train = image_train
X_HA_train = day_train
label_train = label_train

In [8]:
#模型拟合与评估
finish_model.fit([X_train,X_HA_train], label_train, epochs=100, batch_size=128,
validation_data=([image_test,day_test], label_test))
# finish_model.evaluate(image_test, label_test)

Epoch 1/100
8/8 [==============================] - 3s 30ms/step - loss: 3093.4478 - val_loss: 2815.1533
Epoch 2/100
8/8 [==============================] - 0s 7ms/step - loss: 3022.1533 - val_loss: 2576.2319
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 2910.7964 - val_loss: 2310.2041
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 2717.7627 - val_loss: 2087.4944
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 2417.3804 - val_loss: 1731.5836
Epoch 6/100
8/8 [==============================] - 0s 7ms/step - loss: 1991.3341 - val_loss: 1428.4752
Epoch 7/100
8/8 [==============================] - 0s 7ms/step - loss: 1519.2439 - val_loss: 1332.0604
Epoch 8/100
8/8 [==============================] - 0s 7ms/step - loss: 1172.7112 - val_loss: 1320.4541
Epoch 9/100
8/8 [==============================] - 0s 7ms/step - loss: 1000.7850 - val_loss: 1363.0374
Epoch 10/100
8/8 [==============================] - 0s 7ms/step - loss: 

8/8 [==============================] - 0s 7ms/step - loss: 234.2514 - val_loss: 229.5527
Epoch 82/100
8/8 [==============================] - 0s 8ms/step - loss: 225.9239 - val_loss: 211.6688
Epoch 83/100
8/8 [==============================] - 0s 8ms/step - loss: 234.5336 - val_loss: 207.9254
Epoch 84/100
8/8 [==============================] - 0s 7ms/step - loss: 224.5180 - val_loss: 210.1490
Epoch 85/100
8/8 [==============================] - 0s 7ms/step - loss: 221.5730 - val_loss: 236.9770
Epoch 86/100
8/8 [==============================] - 0s 8ms/step - loss: 218.8351 - val_loss: 219.5593
Epoch 87/100
8/8 [==============================] - 0s 7ms/step - loss: 213.8279 - val_loss: 226.6540
Epoch 88/100
8/8 [==============================] - 0s 8ms/step - loss: 211.5372 - val_loss: 206.3027
Epoch 89/100
8/8 [==============================] - 0s 7ms/step - loss: 206.6976 - val_loss: 203.9854
Epoch 90/100
8/8 [==============================] - 0s 7ms/step - loss: 207.7497 - val_loss: 19

In [9]:
#模型预测
model_pre = finish_model.predict([image_test,day_test])

In [11]:
#计算各项误差指标

mape_mean = dan_utils.mape_loss_func(model_pre, label_test)
smape_mean = dan_utils.smape_loss_func(model_pre, label_test)
mae_mean = dan_utils.mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.2642430992187431
smape = 0.24545049368329783
mae = 10.179607212471044


In [12]:
#模型保存
finish_model.save_weights('../model/source_%s.h5'%class_color_set[class_i])


In [14]:
#计算每条路段的误差
mape_list = []
for i in range(num_links):
    a1 = dan_utils.mape_loss_func(model_pre[:,i,:], label_test[:,i,:])
    mape_list.append(a1)
    print(str(i+1)+'th link')
    print(a1)

1th link
0.291418888301914
2th link
0.20615807811446704
3th link
0.142309093239803
4th link
0.29527213239505107
5th link
0.21218772656484716
6th link
0.2613052053722919
7th link
0.3387219361153922
8th link
0.08444143306069736
9th link
0.11022018904201941
10th link
0.2676980011088829
11th link
0.11709705036257698
12th link
0.1987071693718227
13th link
0.43025253051377144
14th link
0.21045421069718948
15th link
0.2802105438373814
16th link
0.27044689879354333
17th link
0.22806834471218507
18th link
0.16005546997562725
19th link
0.3166538336520252
20th link
0.19286563916805596
21th link
0.17696268372718482
22th link
0.13814892339190496
23th link
0.3597297233036539
24th link
1.037468892815995
25th link
0.27922288283029395


In [15]:
mape_pd = pd.Series(mape_list)

In [16]:
mape_pd.sort_values()

7     0.084441
8     0.110220
10    0.117097
21    0.138149
2     0.142309
17    0.160055
20    0.176963
19    0.192866
11    0.198707
1     0.206158
13    0.210454
4     0.212188
16    0.228068
5     0.261305
9     0.267698
15    0.270447
24    0.279223
14    0.280211
0     0.291419
3     0.295272
18    0.316654
6     0.338722
22    0.359730
12    0.430253
23    1.037469
dtype: float64